In [21]:
import pennylane as qml
import numpy as np
from shapely.geometry import Polygon, Point

In [22]:
def I(i):
    return qml.Identity(i)
def X(i):
    return qml.PauliX(i)
def Y(i):
    return qml.PauliY(i)
def Z(i):
    return qml.PauliZ(i)

In [23]:
# Define coordinates of the points of each region# Definir las coordenadas de los puntos de cada región
region01_coords = np.array([(-2, 1), (2, 1), (4, 3), (4, 4), (-4, 4), (-4, 3)])    # Class 0
region02_coords = np.array([(-3, -4), (0, -1), (3, -4)])                           # Class 0
region1_coords = np.array([(0, -1), (3, -4), (4, -4), (4, 3)])                     # Class 1
region2_coords = np.array([(0, -1), (-3, -4), (-4, -4), (-4, 3)])                  # Class 2
region3_coords = np.array([(-2, 1), (2, 1), (0, -1)])                              # Class 3

def labeling(x, y):

    # Create Polygons for each region
    region01_poly = Polygon(region01_coords)
    region02_poly = Polygon(region02_coords)
    region1_poly = Polygon(region1_coords)
    region2_poly = Polygon(region2_coords)
    region3_poly = Polygon(region3_coords)
    
    p = Point(x, y)
    if region01_poly.contains(p):
        return 0
    elif region02_poly.contains(p):
        return 0
    elif region1_poly.contains(p):
        return 1
    elif region2_poly.contains(p):
        return 2
    elif region3_poly.contains(p):
        return 3
    else:
        return None # if the point is not in any region

In [24]:
def get_generators(gen_type): # gen_type referencing at the label of table 1 page 7 paper: https://arxiv.org/pdf/2309.05690.pdf
    
    generators = []
    if gen_type == "a0":    
        # for q in range(nqubits):
        #     m = I(0)
        #     for i in range(nqubits):
        #         if i != q:
        #             m = m @ I(i)
        #         else:
        #             m = m @ Z(q)
        #     generators.append(m)

        for q in range(nqubits-1):
            m = I(0)
            for i in range(nqubits):
                if i != q:
                    m = m @ I(i)
                else:
                    m = m @ X(q) @ X(q+1)
            generators.append(m)
    
    elif gen_type == "a1":
        for q in range(nqubits):
            m = I(0)
            for i in range(nqubits):
                if i != q:
                    m = m @ I(i)
                else:
                    m = m @ Z(q)
            generators.append(m)

        for q in range(nqubits-1):
            m = I(0)
            for i in range(nqubits):
                if i != q:
                    m = m @ I(i)
                else:
                    m = m @ X(q) @ Y(q+1)
            generators.append(m)

    elif gen_type == "a2":
        for q in range(nqubits):
            m = I(0)
            for i in range(nqubits):
                if i != q:
                    m = m @ I(i)
                else:
                    m = m @ Z(q)
            generators.append(m)

        for q in range(nqubits-1):
            m = I(0)
            for i in range(nqubits):
                if i != q:
                    m = m @ I(i)
                else:
                    m = m @ X(q) @ Y(q+1)
            generators.append(m)

        for q in range(nqubits-1):
            m = I(0)
            for i in range(nqubits):
                if i != q:
                    m = m @ I(i)
                else:
                    m = m @ Y(q) @ X(q+1)
            generators.append(m)

    return generators

def pauli_basis(generators):
    zeros = np.zeros((2**nqubits, 2**nqubits))

    def commutator(A,B):
        M = np.matmul(A, B) - np.matmul(B, A)
        # if not (M == zeros).all():
        #     print(np.trace(M*M))
            # M = 1/np.trace(M*M)*M
        return M

    def compare(A,B):
        r = None
        for i in range(len(A)):
            for j in range(len(A[0])):
                if A[i,j] == 0 and B[i,j] == 0:
                    pass
                elif A[i,j] == 0 and B[i,j] != 0 or A[i,j] != 0 and B[i,j] == 0:
                    return False
                elif r is None:
                    r = A[i,j]/B[i,j]
                elif not A[i,j]/B[i,j] == r:
                    return False
        return True

    g_matrices = [qml.matrix(g) for g in generators]
    g_all = g_matrices.copy()
    g_prev = g_matrices.copy()
    num_new = 1

    while num_new != 0:
        g_new = []
        for g in g_matrices:
            for gp in g_prev:
                gn = commutator(g, gp)
                if not (gn == zeros).all():
                    gn_new = True
                    for ga in g_all:
                        if compare(ga,gn):
                            gn_new = False
                            break
                    if gn_new:
                        g_all.append(gn)
                        g_new.append(gn)
        num_new = len(g_new)
        # g_matrices += g_new
        g_prev = g_new

    # print()
    # print(len(g_matrices))
    # print(nqubits,"qubits -", len(g_all), "/", 4**nqubits)

    normalized_basis = []
    for g in g_all:
        normalized_basis.append(1/np.sqrt(np.trace(np.matrix(g).getH() @ g))*g)

        
    # iden = np.zeros((2**nqubits, 2**nqubits))
    # for i in range(2**nqubits):
    #     iden[i,i] = 1/np.sqrt(2**nqubits)
    # normalized_basis.append(iden)


    return normalized_basis

In [25]:
for nqubits in range(3,7):
    # nqubits = 2
    generators = get_generators("a0")
    # print(generators)
    # print(len(generators))
    gen_basis = pauli_basis(generators)
    print(len(gen_basis))
    print(len(gen_basis)/(4**nqubits))

2
0.03125
3
0.01171875
4
0.00390625
5
0.001220703125


In [26]:
def ground_state(j, option):
    
    if option == "ground1":
        hamiltonian = 0
        for i in range(nqubits):
            hamiltonian += j * Z(i)
        for i in range(nqubits-1):
            hamiltonian += X(i) @ X(i+1)
    
    elif option == "ground2":
        hamiltonian = 0
        for i in range(nqubits-1):
            hamiltonian += j * Z(i) @ Z(i+1)
            hamiltonian += X(i) @ X(i+1)
            hamiltonian += Y(i) @ Y(i+1)
    
    elif option == "ground3":
        js = np.random.uniform(-4, 4, (2,))
        hamiltonian = 0
        for i in range(nqubits):
            hamiltonian += Z(i)
            hamiltonian -= js[0] * X(i) @ X((i+1)%nqubits)
            hamiltonian -= js[1] * X((i-1)%nqubits) @ Z(i) @ X((i+1)%nqubits)


    ham_matrix = qml.matrix(hamiltonian)
    _, eigvecs = np.linalg.eigh(ham_matrix)
    
    return eigvecs[:,0]

In [27]:
def qr_haar(N):
    """Generate a Haar-random matrix using the QR decomposition."""
    """https://pennylane.ai/qml/demos/tutorial_haar_measure/"""
    
    A, B = np.random.normal(size=(N, N)), np.random.normal(size=(N, N))
    Z = A + 1j * B

    Q, R = np.linalg.qr(Z)
    Lambda = np.diag([R[i, i] / np.abs(R[i, i]) for i in range(N)])

    return np.dot(Q, Lambda)


def get_random_haar_state(num_qubits):
    qml.QubitUnitary(qr_haar(2**num_qubits), wires=range(num_qubits))
    return qml.state()

In [28]:
def purity(state, gen_basis):
    rho = np.tensordot(state, np.conjugate(state), axes=0)
    # rho = np.zeros((2**nqubits, 2**nqubits))
    # rho[0,0] = 1
    
    pur = 0
    for g in gen_basis:
        plus = np.absolute(np.trace(np.matrix(g).getH() @ rho))**2
        # print(state)
        # # print(rho)
        # print(2**(nqubits/2)*g, plus)
        pur += plus
    # print("done")
    return pur

# Test

In [30]:
# lambs = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.5, 2.0, 5.0, 10.0, 1000.0]
lambs = np.array(list(range(-20, 20, 4)))/10
gen_type = "a0"
states = "ground3"
nqubit_i = 3
nqubit_f = 6

results = np.zeros(shape=(nqubit_f+1 - nqubit_i, len(lambs)))
for i, nqubits in enumerate(list(range(nqubit_i, nqubit_f+1))):
    # print(nqubits)
    generators = get_generators(gen_type)
    gen_basis = pauli_basis(generators)

    for j, lamb in enumerate(lambs):

        if states in ["ground1", "ground2", "ground3"]:
            state = ground_state(lamb, states)
            # print(state)
        elif states == "haar":
            dev_haar = qml.device("default.qubit", wires=nqubits)
            get_random_haar_state2 = qml.QNode(get_random_haar_state, dev_haar)
            state = get_random_haar_state2(nqubits)
            
        elif states == "local_haar":
            dev_haar = qml.device("default.qubit", wires=1)
            get_random_local_haar_state = qml.QNode(get_random_haar_state, dev_haar)
            state = 1
            for _ in range(nqubits):
                state = np.tensordot(state, get_random_local_haar_state(1), axes=0).flatten()
        
        pur = purity(state, gen_basis)
        results[i,j] = pur

# print(np.array_str(results, precision=5))
for i, r in enumerate(results):
    print(i+nqubit_i, np.array_str(r, precision=5))

3 [0.01918 0.02168 0.02777 0.04492 0.04002 0.17702 0.12501 0.02137 0.02194
 0.00169]
4 [1.82286e-01 4.86270e-05 2.83341e-02 1.31836e-01 4.20573e-03 1.05504e-06
 1.20177e-02 1.86912e-01 9.88163e-06 1.14527e-01]
5 [1.01776e-01 9.27436e-04 4.37057e-02 1.23388e-01 2.85928e-02 4.57262e-02
 1.72996e-02 2.86393e-02 2.14341e-02 7.80536e-06]
6 [0.0211  0.00347 0.04265 0.00142 0.03725 0.05036 0.064   0.00015 0.07771
 0.00831]


# Misc

In [79]:
norms = []
for g in gen_basis:
    norms.append(np.sqrt(np.trace(np.matrix(g).getH() @ g)))
print(norms)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j)]


In [230]:
gen_basis = np.array([[[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]])
rho = np.array([[1,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]])
nqubits = 2
rho = np.zeros((2**nqubits, 2**nqubits))
rho[0,0] = 1

gen_basis_norm = []
for g in gen_basis:
    gen_basis_norm.append(1/np.sqrt(np.trace(np.matrix(g).getH() @ g))*g)

pur = 0
for g in gen_basis_norm:
    pur += np.absolute(np.trace(np.matrix(g).getH() @ rho))**2
print(pur)

0.25
